In [7]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Date, Numeric, DateTime
from sqlalchemy.orm import scoped_session, sessionmaker
from sqlalchemy.ext.automap  import automap_base

In [8]:
# The code was removed by Watson Studio for sharing.

In [9]:
#DB connection string
connection_string = "db2+ibm_db://" + user + ":" + pwd + "@dashdb-txn-flex-yp-dal13-43.services.dal.bluemix.net:50000/BLUDB"

In [10]:

engine = create_engine(connection_string, echo=False)
meta = MetaData()

nome_tabela = 'COVID-BR'

covid = Table('COVID-BR', meta, 
    Column("COUNTRY", String(30)),
    Column("PROVINCE_STATE", String(50)),
    Column("CITY", String(50)),
    Column("IBGEID", Integer),
    Column("COD_REGIAODESAUDE",Numeric(18,8)),
    Column("NOME_REGIAODESAUDE",String(50)),
    Column("DEATHS", Integer),
    Column("CASES", Integer),
    Column("DEATHS_PER_100K_INHABITANTS", Numeric(18,8)),
    Column("TOTALCASES_PER_100K_INHABITANTS", Numeric(18,8)),
    Column("DEATHS_BY_TOTALCASES", Numeric(18,8)),
    Column("SOURCE", String(10)),
    Column("DATE", Date),
    Column("NEWCASES", Integer),
    Column("NEWDEATHS",Integer),
    Column("LAST_INFO_DATE", Date))


# date	newCases	newDeaths
#'DEATHS_PER_100K_INHABITANTS','TOTALCASES_PER_100K_INHABITANTS','DEATHS_BY_TOTALCASES'

dropTable = True

if not engine.dialect.has_table(engine, nome_tabela, schema = 'BLUADMIN'):
    meta.create_all(engine)
else:
    if dropTable:
        covid.drop(engine) # Apaga tabela
        meta.create_all(engine) # Cria tabela
    conn = engine.connect()
    #stmt = covid.delete().where(students.c.lastname == 'Khanna')
    stmt = covid.delete()
    conn.execute(stmt)
    conn.close()

In [11]:
Base = automap_base()

#engine = create_engine(connection_string, echo=True)
session = scoped_session(sessionmaker(autocommit=True,
                                      autoflush=False,
                                      bind=engine))

Base.prepare(engine, reflect=True)
base_sa = Base.classes

In [12]:
df = pd.read_csv('https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-cities.csv')
df.head()

country state                    city   ibgeID  cod_RegiaoDeSaude  \
0  Brazil    GO      Abadia de Goiás/GO  5200050            52001.0   
1  Brazil    MG  Abadia dos Dourados/MG  3100104            31074.0   
2  Brazil    GO            Abadiânia/GO  5200100            52011.0   
3  Brazil    PA           Abaetetuba/PA  1500107            15011.0   
4  Brazil    MG               Abaeté/MG  3100203            31024.0   

           name_RegiaoDeSaude  deaths  totalCases  \
0                     Central       1         135   
1  Patrocínio / Monte Carmelo       0          15   
2                    Pirineus       7          77   
3                   Tocantins     109        2598   
4                 Sete Lagoas       0          26   

   deaths_per_100k_inhabitants  totalCases_per_100k_inhabitants  \
0                     11.39861                       1538.81226   
1                      0.00000                        214.62298   
2                     34.92665                        384.19319   
3                     69.11946                       1647.45273   
4                      0.00000                        111.89052   

   deaths_by_totalCases _source        date  newCases  newDeaths  \
0               0.00741     SES  2020-07-25         0          0   
1               0.00000     SES  2020-07-25         0          0   
2               0.09091     SES  2020-07-25         0          0   
3               0.04196     SES  2020-07-25         0          0   
4               0.00000     SES  2020-07-25         0          0   

  last_info_date  
0     2020-07-25  
1     2020-07-24  
2     2020-07-25  
3     2020-07-25  
4     2020-07-24

In [13]:
df.columns = ['COUNTRY','PROVINCE_STATE','CITY',
                     'IBGEID','COD_REGIAODESAUDE','NOME_REGIAODESAUDE','DEATHS','CASES','DEATHS_PER_100K_INHABITANTS',
              'TOTALCASES_PER_100K_INHABITANTS','DEATHS_BY_TOTALCASES','SOURCE','DATE',
              'NEWCASES','NEWDEATHS','LAST_INFO_DATE']
df.head()

COUNTRY PROVINCE_STATE                    CITY   IBGEID  COD_REGIAODESAUDE  \
0  Brazil             GO      Abadia de Goiás/GO  5200050            52001.0   
1  Brazil             MG  Abadia dos Dourados/MG  3100104            31074.0   
2  Brazil             GO            Abadiânia/GO  5200100            52011.0   
3  Brazil             PA           Abaetetuba/PA  1500107            15011.0   
4  Brazil             MG               Abaeté/MG  3100203            31024.0   

           NOME_REGIAODESAUDE  DEATHS  CASES  DEATHS_PER_100K_INHABITANTS  \
0                     Central       1    135                     11.39861   
1  Patrocínio / Monte Carmelo       0     15                      0.00000   
2                    Pirineus       7     77                     34.92665   
3                   Tocantins     109   2598                     69.11946   
4                 Sete Lagoas       0     26                      0.00000   

   TOTALCASES_PER_100K_INHABITANTS  DEATHS_BY_TOTALCASES SOURCE        DATE  \
0                       1538.81226               0.00741    SES  2020-07-25   
1                        214.62298               0.00000    SES  2020-07-25   
2                        384.19319               0.09091    SES  2020-07-25   
3                       1647.45273               0.04196    SES  2020-07-25   
4                        111.89052               0.00000    SES  2020-07-25   

   NEWCASES  NEWDEATHS LAST_INFO_DATE  
0         0          0     2020-07-25  
1         0          0     2020-07-24  
2         0          0     2020-07-25  
3         0          0     2020-07-25  
4         0          0     2020-07-24

In [14]:
nome_tabela = 'COVID-BR'

In [15]:
df.to_sql(name=nome_tabela, con=engine, if_exists='append', index=False)
#df.to_sql(name=nome_tabela, con=engine, if_exists='append', index=False, chunksize=1000)   # With number of records to be processed each time

In [16]:
df2 = pd.read_sql_table(nome_tabela, engine)
df2.head()

country province_state                    city   ibgeid  cod_regiaodesaude  \
0  Brazil             GO      Abadia de Goiás/GO  5200050            52001.0   
1  Brazil             MG  Abadia dos Dourados/MG  3100104            31074.0   
2  Brazil             GO            Abadiânia/GO  5200100            52011.0   
3  Brazil             PA           Abaetetuba/PA  1500107            15011.0   
4  Brazil             MG               Abaeté/MG  3100203            31024.0   

           nome_regiaodesaude  deaths  cases  deaths_per_100k_inhabitants  \
0                     Central       1    135                     11.39861   
1  Patrocínio / Monte Carmelo       0     15                      0.00000   
2                    Pirineus       7     77                     34.92665   
3                   Tocantins     109   2598                     69.11946   
4                 Sete Lagoas       0     26                      0.00000   

   totalcases_per_100k_inhabitants  deaths_by_totalcases SOURCE       DATE  \
0                       1538.81226               0.00741    SES 2020-07-25   
1                        214.62298               0.00000    SES 2020-07-25   
2                        384.19319               0.09091    SES 2020-07-25   
3                       1647.45273               0.04196    SES 2020-07-25   
4                        111.89052               0.00000    SES 2020-07-25   

   newcases  newdeaths last_info_date  
0         0          0     2020-07-25  
1         0          0     2020-07-24  
2         0          0     2020-07-25  
3         0          0     2020-07-25  
4         0          0     2020-07-24

In [17]:
session.close()